In [56]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import sqrt
from scipy.stats import pearsonr

%matplotlib inline

In [2]:
#------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------
# Content-Based recommendation system
#------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------

In [3]:
# Download the movie data set
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')

# Unpack the data set
!unzip -o -j moviedataset.zip -d data/

# Remove the archive
!rm -f moviedataset.zip

--2020-02-14 12:00:49--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: 'moviedataset.zip'

moviedataset.zip    100%[===================>] 152.88M  10.6MB/s    in 15s     

2020-02-14 12:01:06 (10.1 MB/s) - 'moviedataset.zip' saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: data/links.csv          
  inflating: data/movies.csv         
  inflating: data/ratings.csv        
  inflating: data/README.txt         
  inflating: data/tags.csv           


In [4]:
# Load the csv files
movies_df = pd.read_csv('data' + os.path.sep + 'movies.csv', index_col = False)
display(movies_df.head())
ratings_df = pd.read_csv('data' + os.path.sep + 'ratings.csv', index_col = False)
display(ratings_df.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [5]:
# Remove the year from the title column by using pandas'
# replace function and store in a new year column.
print('Movies df shape:', movies_df.shape)

# Extract the year - the year in round brackets
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand = False)
display(movies_df.year.head())

# Remove the round brackets from the year
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand = False)
display(movies_df.year.head())

# Remove the year from the title
movies_df.title = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
display(movies_df.title.head())

# Strip the title
movies_df.title = movies_df.title.apply(lambda x: x.strip())
display(movies_df.title.head())

# See id the re is any NA
print('Number of NULL year movies, before:', movies_df[movies_df.isnull().year].shape[0])
movies_df.loc[movies_df.isnull().year,'year'] = '1900'
print('Number of NULL year movies, after:', movies_df[movies_df.isnull().year].shape[0])

Movies df shape: (34208, 3)


0    (1995)
1    (1995)
2    (1995)
3    (1995)
4    (1995)
Name: year, dtype: object

0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: object

0                      Toy Story 
1                        Jumanji 
2               Grumpier Old Men 
3              Waiting to Exhale 
4    Father of the Bride Part II 
Name: title, dtype: object

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

Number of NULL year movies, before: 68
Number of NULL year movies, after: 0


In [6]:
# Check on the column types
print(movies_df.dtypes)

# Convert the years column into numeric
movies_df.year = movies_df.year.astype('int32')

print(movies_df.dtypes)

movieId     int64
title      object
genres     object
year       object
dtype: object
movieId     int64
title      object
genres     object
year        int32
dtype: object


In [7]:
# Split the values in the Genres column into a list of Genres to simplify future use
movies_df.genres = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [8]:
# Keeping genres in a list format isn't optimal for the content-based
# recommendation system technique, we will use the One Hot Encoding
# technique to convert the list of genres to a vector where each
# column corresponds to one possible value of the feature. This
# encoding is needed for feeding categorical data. In this case,
# we store every different genre in columns that contain either 1
# or 0. 1 shows that a movie has that genre and 0 shows that it doesn't.
movies_wg_df = movies_df.copy()

# Iterate over the genres in each row and add flags as columns
for idx, row in movies_df.iterrows():
    for genre in row['genres']:
        movies_wg_df.at[idx, genre] = 1

# The resulting N/A entries in the newly added colums are to be filled with zeros
movies_wg_df.fillna(0, inplace = True)
movies_wg_df = movies_wg_df.drop(['genres'], axis = 1)
movies_wg_df.head()

,movieId,title,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# We won't be needing the timestamp column, so let's drop it to save on memory.
ratings_df = ratings_df.drop('timestamp', 1)
display(ratings_df.head())

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [10]:
# Content-Based or Item-Item recommendation system: 
#     Attempts to figure out what a user's favourite aspects of an item is,
#     and then recommends items that present those aspects. In our case,
#     we're going to try to figure out the input's favorite genres from
#     the movies and ratings given.

# Take ratings for the user with id 5
user_id = 5
user_r_df = ratings_df[ratings_df.userId == user_id]
display(user_r_df.head())

# Add the movie title to this user input
user_movies = movies_df[movies_df.movieId.isin(user_r_df.movieId.tolist())]
display(user_movies.head())

# Merge the title column into the user ratings
user_r_df = pd.merge(user_r_df, user_movies)
user_r_df = user_r_df.drop(['genres', 'year'], 1)
display(user_r_df.head())


,userId,movieId,rating
194,5,1203,4.0
195,5,2571,2.5
196,5,6016,5.0
197,5,7502,4.5
198,5,58559,4.0


,movieId,title,genres,year
1178,1203,12 Angry Men,[Drama],1957
2487,2571,"Matrix, The","[Action, Sci-Fi, Thriller]",1999
5918,6016,City of God (Cidade de Deus),"[Action, Adventure, Crime, Drama, Thriller]",2002
7357,7502,Band of Brothers,"[Action, Drama, War]",2001
12540,58559,"Dark Knight, The","[Action, Crime, Drama, IMAX]",2008


,userId,movieId,rating,title
0,5,1203,4.0,12 Angry Men
1,5,2571,2.5,"Matrix, The"
2,5,6016,5.0,City of God (Cidade de Deus)
3,5,7502,4.5,Band of Brothers
4,5,58559,4.0,"Dark Knight, The"


In [11]:
# Let us now start by learning the user's preferences. We'll only need the actual
# genre table, so let's clean this up a bit by resetting the index and dropping 
# the movieId, title, genres and year columns.
user_movies_pref = movies_wg_df[movies_wg_df.movieId.isin(user_r_df.movieId.tolist())]
user_movies_pref = user_movies_pref.drop(['movieId', 'year', 'title', '(no genres listed)'], 1)
user_movies_pref = user_movies_pref.reset_index(drop = True)
display(user_movies_pref.head())

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# To turn each genre into a weight we should take the user's review ratings
# and multiplying them into the user's genre table and then summing up the
# resulting table by column. which is a dot product.
user_profile = user_movies_pref.transpose().dot(user_r_df['rating'])
print(user_profile)
print('User profile shape:', user_profile.shape)

Adventure      16.0
Animation       3.0
Children        3.0
Comedy         25.5
Fantasy         3.0
Romance         1.5
Drama          59.5
Action         41.0
Crime          21.0
Thriller       32.5
Horror          0.0
Mystery         4.5
Sci-Fi         16.0
IMAX            7.0
Documentary     0.0
War            15.0
Musical         0.0
Western         0.0
Film-Noir       0.0
dtype: float64
User profile shape: (19,)


In [13]:
# Using the user profile, we can recommend movies that satisfy the user's preferences.
movies_wg_df.set_index(movies_wg_df.movieId, inplace= True)
all_movie_genres = movies_wg_df.drop('movieId', 1).drop('title', 1).drop('year', 1).drop('(no genres listed)', 1)
display(all_movie_genres.head())
print('All movie genres:', all_movie_genres.shape)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir
movieId,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


All movie genres: (34208, 19)


In [14]:
# Now to compute the user preferences for each movie:
#Multiply the genres by the weights and then take the weighted average
user_movie_prefs = ((all_movie_genres * user_profile).sum(axis=1)) / (user_profile.sum())
print('user_movie_prefs type:', type(user_movie_prefs))
user_movie_prefs = pd.DataFrame( {'userScore': user_movie_prefs})
display(user_movie_prefs.head())

user_movie_prefs type: <class 'pandas.core.series.Series'>


,userScore
movieId,
1,0.203219
2,0.088531
3,0.108652
4,0.348089
5,0.102616


In [15]:
# Restore the movies
user_movie_final = pd.merge(user_movie_prefs, movies_df, left_index=True, right_on='movieId').sort_values(by='userScore', ascending=False)
display(user_movie_final.head())

,userScore,movieId,title,genres,year
24565,0.808853,115479,"Whip Hand, The","[Action, Adventure, Crime, Drama, Sci-Fi, Thri...",1951
16055,0.804829,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
13250,0.786720,64645,The Wrecking Crew,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1968
26442,0.786720,122787,The 39 Steps,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1959
15001,0.758551,75408,Lupin III: Sweet Lost Night (Rupan Sansei: Swe...,"[Action, Animation, Comedy, Crime, Drama, Myst...",2008


In [16]:
# We can also just compute the top 20 scoring movies for the user:
top_20_user_scores = user_movie_prefs.sort_values(by='userScore', ascending=False).head(20)
top_20_user_movies = movies_df[movies_df['movieId'].isin(top_20_user_scores.index)]
top_20_user_movies = pd.merge(top_20_user_scores, top_20_user_movies, left_index = True, right_on = 'movieId')
display(top_20_user_movies)

,userScore,movieId,title,genres,year
24565,0.808853,115479,"Whip Hand, The","[Action, Adventure, Crime, Drama, Sci-Fi, Thri...",1951
16055,0.804829,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
13250,0.786720,64645,The Wrecking Crew,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1968
26442,0.786720,122787,The 39 Steps,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1959
15001,0.758551,75408,Lupin III: Sweet Lost Night (Rupan Sansei: Swe...,"[Action, Animation, Comedy, Crime, Drama, Myst...",2008
15073,0.758551,76153,Lupin III: First Contact (Rupan Sansei: Faasut...,"[Action, Animation, Comedy, Crime, Drama, Myst...",2002
11494,0.744467,49530,Blood Diamond,"[Action, Adventure, Crime, Drama, Thriller, War]",2006
4625,0.740443,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
15562,0.730382,79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010
27514,0.728370,127341,Longshot,"[Action, Comedy, Crime, Drama, Romance, Thriller]",2001


In [17]:
#------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------
# Collaborative Filtering
#------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------

In [18]:
# Here we can use the same movie ratings data set as before

# We can drop the genres column as it is not used in this type of recommender system
movies_df = movies_df.drop('genres', 1)
display(movies_df.head())

# The ratings data frame can be used as is
display(ratings_df.head())

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [19]:
# Collaborative Filtering, also known as User-User Filtering:
# 
# As hinted by its alternate name, this technique uses other users to recommend 
# items to the input user. It attempts to find users that have similar preferences
# and opinions as the input user and then recommends items that they have liked to
# the input user. There are several methods of finding similar users (Even some
# making use of Machine Learning). The one we will be using here is going to be
# based on the Pearson Correlation Function.
#
# The process for creating a User Based recommendation system is as follows:
#
# 1. Select a user with the movies the user has watched
# 2. Based on his rating to movies, find the top X neighbours
# 3. Get the watched movie record of the user for each neighbour.
# 4. Calculate a similarity score using some formula
# 5. Recommend the items with the highest score

# We can use the same user 5 ratings as before
display(user_r_df.head())

,userId,movieId,rating,title
0,5,1203,4.0,12 Angry Men
1,5,2571,2.5,"Matrix, The"
2,5,6016,5.0,City of God (Cidade de Deus)
3,5,7502,4.5,Band of Brothers
4,5,58559,4.0,"Dark Knight, The"


In [44]:
# Let us now find the other users who have seen the same movies
same_movie_ratings = ratings_df[ratings_df['movieId'].isin(user_r_df['movieId'].tolist())]
display(same_movie_ratings.head(20))
print('Same moview ratings shape: ', same_movie_ratings.shape)

,userId,movieId,rating
3,2,2571,3.5
5,2,112552,5.0
6,2,112556,4.0
105,4,2571,4.0
194,5,1203,4.0
195,5,2571,2.5
196,5,6016,5.0
197,5,7502,4.5
198,5,58559,4.0
199,5,79132,3.0


Same moview ratings shape:  (207240, 3)


In [54]:
# Remove the input user from the list
others_movie_ratings = same_movie_ratings[same_movie_ratings['userId'] != 5]

# Group by user id and select a user with is 1130
user_groups = others_movie_ratings.groupby(['userId'])

user_groups.get_group(1130)

,userId,movieId,rating
104530,1130,2571,2.0
104789,1130,6016,4.0
104998,1130,58559,3.0
105069,1130,79132,2.5
105155,1130,119155,3.0
105156,1130,122882,1.0


In [55]:
# Let's also sort these groups so the users that share the 
# most movies in common with the input have higher priority. 

sorted_user_groups = sorted(user_groups, key = lambda x : len(x[1]), reverse = True)
display(sorted_user_groups[0:3])

[(99836,          userId  movieId  rating
  9243129   99836     1203     4.5
  9243403   99836     2571     3.5
  9243812   99836     6016     3.5
  9243919   99836     7502     3.5
  9244131   99836    58559     3.5
  9244222   99836    79132     5.0
  9244312   99836    92259     3.5
  9244447   99836   112183     4.5
  9244450   99836   112552     4.0
  9244451   99836   112556     4.0
  9244462   99836   114060     4.0
  9244463   99836   114180     2.0
  9244469   99836   114662     4.0
  9244471   99836   115170     3.0
  9244472   99836   115210     2.5
  9244475   99836   115617     3.0
  9244477   99836   115713     3.5
  9244480   99836   116797     3.5
  9244495   99836   119141     1.0
  9244496   99836   119145     3.5
  9244497   99836   119155     2.0
  9244498   99836   120466     2.5
  9244505   99836   122882     3.0
  9244515   99836   129354     3.0), (105787,          userId  movieId  rating
  9800096  105787     1203     5.0
  9800355  105787     2571     5.0
  98

In [65]:
# Find out how similar each user is to the input through the Pearson Correlation Coefficient
#
# Why Pearson Correlation?
#
# Pearson correlation is invariant to scaling, i.e. multiplying all elements by 
# a nonzero constant or adding any constant to all elements. For example, if you
# have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is
# a pretty important property in recommendation systems because for example two 
# users might rate two series of items totally different in terms of absolute
# rates, but they would be similar users (i.e. with similar ideas) with similar
# rates in various scales .
# 
# The correlation values varry from -1 to 1, where:
# * '1'  -- forms a direct correlation between the two entities
#           it means a perfect positive correlation
# * '-1' -- forms a perfect negative correlation.
#
# In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.
#

# Sort the input user movie ratings by the movie id
user_r_df = user_r_df.sort_values(by='movieId')

#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearson_corr = {}

for name, group in sorted_user_groups[0:100]:
    other_ratings = group.sort_values(by='movieId').rating.values
    user_ratings = user_r_df.loc[user_r_df.movieId.isin(group.movieId.tolist()), 'rating'].values
    corr = pearsonr(other_ratings, user_ratings)
    pearson_corr[name] = corr[0]
    #print('pearson_corr for', name, 'is', corr[0])

In [77]:
# Create the user correlations data frame
user_corr = pd.DataFrame.from_dict(pearson_corr, orient = 'index')
user_corr.columns = ['corrScore']
user_corr['userId'] = user_corr.index
user_corr.index = range(len(pearson_corr))
display(user_corr.head())

# Only keep the positive correlations
print('user_corr before:', user_corr.shape[0])
user_corr = user_corr[user_corr.corrScore > 0]
print('user_corr after:', user_corr.shape[0])

# Order by the score, the higher the better
user_corr = user_corr.sort_values('corrScore', ascending = False)
display(user_corr.head())

# Choose the top 30 users
top_30_corr_isers = user_corr.head(30)

,corrScore,userId
0,0.437287,99836
1,0.400310,105787
2,0.275943,135635
3,0.498380,215174
4,0.431074,57603


user_corr before: 100
user_corr after: 87


,corrScore,userId
96,0.768995,77917
62,0.723990,78242
36,0.717447,107714
35,0.679770,76683
64,0.657061,132202


In [78]:
# Now combine the scores with the movie ratings, using the inner joint
others_movie_scores = pd.merge(ratings_df, top_30_corr_isers, left_on='userId', right_on='userId', how='inner')
others_movie_scores.head()

,userId,movieId,rating,corrScore
0,2824,1,4.5,0.468806
1,2824,6,4.5,0.468806
2,2824,10,4.0,0.468806
3,2824,29,4.5,0.468806
4,2824,32,4.5,0.468806


In [100]:
# Multiply the correlation with the movie rating
others_movie_scores['corrRatings'] = others_movie_scores['rating']*others_movie_scores['corrScore']
others_movie_scores.head()

,userId,movieId,rating,corrScore,corrRatings
0,2824,1,4.5,0.468806,2.109627
1,2824,6,4.5,0.468806,2.109627
2,2824,10,4.0,0.468806,1.875224
3,2824,29,4.5,0.468806,2.109627
4,2824,32,4.5,0.468806,2.109627


In [102]:
# Now compute the mean movie scores
sim_user_score = others_movie_scores.groupby('movieId').sum()[['corrRatings', 'corrScore']]
sim_user_score['simUserScore'] = sim_user_score['corrRatings'] / sim_user_score['corrScore']
sim_user_score['movieId'] = sim_user_score.index
sim_user_score.index = range(len(sim_user_score))

# Drop the unneeded columns
sim_user_score = sim_user_score.drop(['corrRatings', 'corrScore'], 1)
sim_user_score = sim_user_score.sort_values(by='simUserScore', ascending = False)

# Display the top and bottom 10
display(sim_user_score.head(10))
display(sim_user_score.tail(10))

top_10_recommend = sim_user_score.head(10)

,simUserScore,movieId
652,5.0,1163
8606,5.0,148881
3375,5.0,6581
7702,5.0,108715
7658,5.0,107643
2704,5.0,4928
7668,5.0,107743
3614,5.0,7074
8320,5.0,130351
4443,5.0,27722


,simUserScore,movieId
5065,0.5,47830
5387,0.5,54290
473,0.5,810
8258,0.5,128550
4966,0.5,45259
1488,0.5,2555
2559,0.5,4598
3913,0.5,8131
7298,0.5,99371
4874,0.5,42783


In [105]:
# Now display the top 10 movies to be recommended
top_10_rec = pd.merge(movies_df.loc[movies_df['movieId'].isin(top_10_recommend['movieId'].tolist())], top_10_recommend)
top_10_rec.rename(columns = {'simUserScore':'Score'}, inplace = True)
top_10_rec

,movieId,title,year,Score
0,1163,Mina Tannenbaum,1994,5.0
1,4928,That Obscure Object of Desire (Cet obscur obje...,1977,5.0
2,6581,"Private Life of Sherlock Holmes, The",1970,5.0
3,7074,"Navigator, The",1924,5.0
4,27722,Last Life in the Universe (Ruang rak noi nid m...,2003,5.0
5,107643,Here Without Me (Inja bedoone man),2011,5.0
6,107743,"Class of 92, The",2013,5.0
7,108715,Better Living Through Chemistry,2014,5.0
8,130351,The Wrecking Crew,2008,5.0
9,148881,World of Tomorrow,2015,5.0


In [ ]:
# Advantages and Disadvantages of Collaborative Filtering
#
# Advantages
#
# * Takes other user's ratings into consideration
# * Doesn't need to study or extract information from the recommended item
# * Adapts to the user's interests which might change over time
#
# Disadvantages
# 
# * Approximation function can be slow
# * There might be a low of amount of users to approximate
# * Privacy issues when trying to learn the user's preference